In [0]:
%sql
WITH opco_mapping AS (
  SELECT 
    a.*,
    TRIM(d.opco_name) AS opco_name_trimmed,
    CASE 
      WHEN TRIM(d.opco_name) IN ('Giant-Carlisle', 'Martins') THEN 1002
      WHEN TRIM(d.opco_name) = 'Stop & Shop' THEN 1001
      WHEN TRIM(d.opco_name) = 'Giant-Landover' THEN 1000
      WHEN TRIM(d.opco_name) = 'Food Lion' THEN 1003
      ELSE 9999 
    END AS Banner_ID_1,
    CASE 
      WHEN TRIM(d.opco_name) IN ('Giant-Carlisle', 'Martins') THEN 'TGC'
      WHEN TRIM(d.opco_name) = 'Stop & Shop' THEN 'SS'
      WHEN TRIM(d.opco_name) = 'Giant-Landover' THEN 'GF'
      WHEN TRIM(d.opco_name) = 'Food Lion' THEN 'FL'
      ELSE '' 
    END AS Banner1,
    CASE 
      WHEN TRIM(d.opco_name) = 'Giant-Carlisle' THEN 'TGC_Giant'
      WHEN TRIM(d.opco_name) = 'Martins' THEN 'TGC_Martins'
      WHEN TRIM(d.opco_name) = 'Stop & Shop' THEN 'SS'
      WHEN TRIM(d.opco_name) = 'Giant-Landover' THEN 'GF'
      WHEN TRIM(d.opco_name) = 'Food Lion' THEN 'FL'
      ELSE '' 
    END AS Banner2,
    CASE 
      WHEN a.dlv_meth_cd = 'P' THEN 'Pickup'
      ELSE 'Delivery' 
    END AS Channel,
    CASE 
      WHEN a.ord_seq_num = 1 THEN 'N'
      ELSE 'E' 
    END AS Cust_Exp,
    IFNULL(b.store_nbr, INT(e.stor_nbr_cd)) store_nbr,
    a.pup_id,
    a.stor_id
  FROM merchandising.pos.rdt_ord_sum_cdm a
  JOIN merchandising.pos.rdt_ecom_xref_cdm d ON TRIM(a.stor_id) = d.stor_id
  left join (select distinct pup_id, stor_id, INT(ahld_stor_nbr_cd) store_nbr from managed_assets.location.cit_pup_ctl_sdm where ahld_stor_nbr_cd != 'NULL') b on a.pup_id = b.pup_id and a.stor_id = b.stor_id
  left join managed_assets.location.cit_zone_ctl_sdm e on a.stor_id = trim(e.stor_id)
  WHERE a.dlv_dt_src >= '2021-01-03' AND a.it_dlv_qy IS NOT NULL
),
snap_data AS (
  SELECT 
    ord_id, 
    COUNT(DISTINCT it_cd) AS ReasonCode_Count, 
    SUM(pr_qy) AS SNAP_Dollars
  FROM  merchandising.pos.art_ord_dtl_cdm
  WHERE it_cd IN ('N-560', 'N-561') AND arc_ld_dt_src > '2021-01-01'
  GROUP BY ord_id
)
SELECT 
  Banner_ID_1 as Banner_ID,
  Banner1,
  Banner2,
  'Prism' AS Platform,
  'Prism' AS Site,
  Channel,
  dlv_dt_src AS Dlvry_Date,
  Cust_Exp,
  store_nbr,
  pup_id,
  stor_id,
  COUNT(DISTINCT a.ord_id) AS Total_Orders,
  ROUND(SUM(a.it_dlv_qy), 2) AS Total_Sales,
  COUNT(DISTINCT b.ord_id) AS SNAP_Orders,
  ROUND(SUM(CASE WHEN b.ord_id IS NULL THEN 0 ELSE a.it_dlv_qy END), 2) AS SNAP_Basket_Sales,
  ABS(ROUND(SUM(CASE WHEN b.ord_id IS NULL THEN 0 ELSE b.SNAP_Dollars END), 2)) AS SNAP_Dollars
FROM opco_mapping a
LEFT JOIN snap_data b ON a.ord_id = b.ord_id
where store_nbr IS NULL
GROUP BY 
 Banner_ID_1,
  Banner1,
  Banner2,
  Channel,
  dlv_dt_src,
  Cust_Exp,
  store_nbr,
  pup_id,
  stor_id

In [0]:
WITH opco_mapping AS (
  SELECT 
    a.*,
    TRIM(d.opco_name) AS opco_name_trimmed,
    CASE 
      WHEN TRIM(d.opco_name) IN ('Giant-Carlisle', 'Martins') THEN 1002
      WHEN TRIM(d.opco_name) = 'Stop & Shop' THEN 1001
      WHEN TRIM(d.opco_name) = 'Giant-Landover' THEN 1000
      WHEN TRIM(d.opco_name) = 'Food Lion' THEN 1003
      ELSE 9999 
    END AS Banner_ID_1,
    CASE 
      WHEN TRIM(d.opco_name) IN ('Giant-Carlisle', 'Martins') THEN 'TGC'
      WHEN TRIM(d.opco_name) = 'Stop & Shop' THEN 'SS'
      WHEN TRIM(d.opco_name) = 'Giant-Landover' THEN 'GF'
      WHEN TRIM(d.opco_name) = 'Food Lion' THEN 'FL'
      ELSE '' 
    END AS Banner1,
    CASE 
      WHEN TRIM(d.opco_name) = 'Giant-Carlisle' THEN 'TGC_Giant'
      WHEN TRIM(d.opco_name) = 'Martins' THEN 'TGC_Martins'
      WHEN TRIM(d.opco_name) = 'Stop & Shop' THEN 'SS'
      WHEN TRIM(d.opco_name) = 'Giant-Landover' THEN 'GF'
      WHEN TRIM(d.opco_name) = 'Food Lion' THEN 'FL'
      ELSE '' 
    END AS Banner2,
    CASE 
      WHEN a.dlv_meth_cd = 'P' THEN 'Pickup'
      ELSE 'Delivery' 
    END AS Channel,
    CASE 
      WHEN a.ord_seq_num = 1 THEN 'N'
      ELSE 'E' 
    END AS Cust_Exp,
    IFNULL(b.store_nbr, INT(e.stor_nbr_cd)) store_nbr,
    a.pup_id,
    a.stor_id
  FROM merchandising.pos.rdt_ord_sum_cdm a
  JOIN merchandising.pos.rdt_ecom_xref_cdm d ON TRIM(a.stor_id) = d.stor_id
  left join (select distinct pup_id, stor_id, INT(ahld_stor_nbr_cd) store_nbr from managed_assets.location.cit_pup_ctl_sdm where ahld_stor_nbr_cd != 'NULL') b on a.pup_id = b.pup_id and a.stor_id = b.stor_id
  left join managed_assets.location.cit_zone_ctl_sdm e on a.stor_id = trim(e.stor_id)
  WHERE a.dlv_dt_src >= '2021-01-03' AND a.it_dlv_qy IS NOT NULL
)
SELECT * FROM opco_mapping where store_nbr is null

In [0]:
%sql
select * from managed_assets.location.cit_pup_ctl_sdm
where pup_id = 10117